In [4]:
#Import libraries
import math
import numpy as np
from scipy.stats import norm

def newton_step(f, x0):
    def df(x):
        dx = 0.00001
        return (f(x + dx) - f(x)) / dx
    return x0 - f(x0)/df(x0)

def newton(f,x0,tol):
    while (abs(newton_step(f, x0) - x0) > tol):
        x0 = newton_step(f, x0)
    return x0

In [8]:
def CallPrice(S, sigma, K, T, r):
    d1 = (math.log(S / K) + (r + .5 * sigma**2) * T) / (sigma * T**.5)
    d2 = d1 - sigma * T**0.5
    n1 = norm.cdf(d1)
    n2 = norm.cdf(d2)
    DF = math.exp(-r * T)
    price=S * n1 - K * DF * n2
    return price

In [19]:
CallPriceVol = lambda vol: CallPrice(S, vol, K, T, r) - C
#parameters
S = 226.5 #asset price
K = 225 #strike price
T = 30/360 #time to maturity
r = .0533 #risk-free interest rate
vol = .2 #implied volatility
C = 8.65 #CallPrice(S, vol, K, T, r) #target priceb

In [20]:
init = .1
tol = 10**-8
print(newton(CallPriceVol, init, tol))

0.28279437261852686


In [21]:
x0 = init
for i in range(0, 4):
    print(x0)
    x0 = newton_step(CallPriceVol, x0)

0.1
0.29168324741347657
0.28279655868098896
0.28279437261852686


In [22]:
def InflexionPoint(S, K, T, r):
    m = S / (K * math.exp(-r * T))
    return math.sqrt(2 * np.abs(math.log(m)) / T)

I = InflexionPoint(S, K, T, r)
print("Inflexion Point:",I)

Inflexion Point: 0.5158187910962976


In [28]:
def vega(S, sigma, K, T, r):
    d1 = (math.log(S / K) + (r + .5 * sigma**2) * T) / (sigma * T**.5)
    vega = S * T**0.5 * norm.pdf(d1)
    return vega

def ImpliedVolCall(C, S, K, r, T,tol):
    x0 = InflexionPoint(S, K, T, r)
    p = CallPrice(S, x0, K, T, r)
    v = vega(S, x0, K, T, r)
    print('c', p-C, 'v',v)
    while (abs((p - C) / v) > tol):
        x0 = x0 - (p - C) / v
        p = CallPrice(S, x0, K, T, r)
        v = vega(S, x0, K, T, r)
        print('c',p-C,'v', v)
    return x0

tol= 10**-8
ImpliedVolCall(C, S, K, r, T,tol)

c 6.004056850607162 v 25.7972236955849
c 0.007293287467410536 v 25.681549024903862
c 6.129406493471379e-08 v 25.6811171161958


0.28279437500447296

In [31]:
CallPrice(S, 0.3, K, T, r)

9.092070621969285